<h1>Data prep and merge</h1>

<h2>Install required package/libraries and directory</h2>

In [283]:

# If not already installed, install pyreadstat (used to read .sas7bdat files)
!pip install pyreadstat
!pip install openpyxl

# Import core packages
import pandas as pd
import os
import numpy as np
import pyreadstat
from scipy.stats import sem
import statsmodels.api as sm
import warnings
warnings.filterwarnings("ignore")


#path 
output_path = "/Users/dengshuyue/Desktop/SDOH/analysis/output/demo_summary.csv"
data_path = "/Users/dengshuyue/Desktop/SDOH/analysis/data"


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


<h2>Load and preview SAS dataset</h2>

In [307]:
# Path to your .sas7bdat file
file_path = "/Users/dengshuyue/Desktop/SDOH/analysis/data/less_important/demo_i.sas7bdat"

# Load the dataset using pandas (requires pyreadstat)
df = pd.read_sas(file_path, format="sas7bdat")

# Preview the data
pd.set_option('display.max_columns', None)
df.head()

# List all column names
print(df.columns.tolist())

['SEQN', 'SDDSRVYR', 'RIDSTATR', 'RIAGENDR', 'RIDAGEYR', 'RIDAGEMN', 'RIDRETH1', 'RIDRETH3', 'RIDEXMON', 'RIDEXAGM', 'DMQMILIZ', 'DMQADFC', 'DMDBORN4', 'DMDCITZN', 'DMDYRSUS', 'DMDEDUC3', 'DMDEDUC2', 'DMDMARTL', 'RIDEXPRG', 'SIALANG', 'SIAPROXY', 'SIAINTRP', 'FIALANG', 'FIAPROXY', 'FIAINTRP', 'MIALANG', 'MIAPROXY', 'MIAINTRP', 'AIALANGA', 'DMDHHSIZ', 'DMDFMSIZ', 'DMDHHSZA', 'DMDHHSZB', 'DMDHHSZE', 'DMDHRGND', 'DMDHRAGE', 'DMDHRBR4', 'DMDHREDU', 'DMDHRMAR', 'DMDHSEDU', 'WTINT2YR', 'WTMEC2YR', 'SDMVPSU', 'SDMVSTRA', 'INDHHIN2', 'INDFMIN2', 'INDFMPIR']


In [ ]:

# Path to your .sas7bdat file
file_path = "/Users/dengshuyue/Desktop/SDOH/analysis/data/less_important/gg.sas7bdat"

# Load the dataset using pandas (requires pyreadstat)
df = pd.read_sas(file_path, format="sas7bdat")

# Preview the data
pd.set_option('display.max_columns', None)
df.head()

# List all column names
print(df.columns.tolist())


In [308]:

# Path to your .sas7bdat file
file_path = "/Users/dengshuyue/Desktop/SDOH/analysis/data/sodh_diet_mort.sas7bdat"

# Load the dataset using pandas (requires pyreadstat)
df = pd.read_sas(file_path, format="sas7bdat")

# Preview the data
pd.set_option('display.max_columns', None)
df.head()

# List all column names
# print(df.columns.tolist())

,SEQN,marriage,HOD050,HOQ065,EMPLOY,unemployment,ELIGSTAT,MORTSTAT,UCOD_LEADING,DIABETES,HYPERTEN,PERMTH_INT,PERMTH_EXM,Death_heart,Death_cancer,Death_resp,Death_cerev,Death_diabe,Death_other,death_cvd,death_cmd,FSDHH,SNAP,FS,ins,RIDAGEYR,INDFMPIR,diabe,smk,smk_avg,sex,race,HEI2015_TOTAL_SCORE,edu,alcg2,DAYS,WTDRD1,DR12DRST,WTDR2D,i_optup,i_FCS,i_HSR,i_nutri,perE_alco,sdmvpsu,sdmvstra,tchol,hdl,ldl,tg,bmi,dm_self,hba1c,cancer,sbp,dbp,dm_rx,chol_rx,angina_rx,CVD,lung_disease,angina,met_hr,wt10,wt,i_FCS_sd,i_Optup_sd,i_nutri_sd,i_HSR_sd,hei2015_sd,Death_inj,Death_alz,Death_infl,Death_kid,death_other1,Death_oth2,death_cmdk,death_cmdkh,death_multi,agesq,py,agestart,ageend,pir,bmic,include,ins2,unemployment2
0,21009.0,1.0,6.0,1.0,1.0,0.0,1.0,0.0,NaN,NaN,NaN,196.0,195.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,55.0,3.79,0.0,1.0,0.0,1.0,1.0,45.542908,2.0,1.0,1.0,56998.593412,1.0,170262.470827,42.652217,25.525713,4.324399,7.838628,0.000000,2.0,31.0,254.0,37.0,176.0,198.0,31.26,2.0,5.9,0.0,120.000000,86.000000,0.0,0.0,0.0,0.0,0.0,0.0,16.000000,5699.859341,21282.808853,2.343959,5.220590,-2.472753,4.281583,3.503301,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3025.0,16.250000,55.0,71.250000,3.0,3.0,1.0,1.0,0.0
1,21010.0,1.0,3.0,1.0,2.0,1.0,1.0,0.0,NaN,NaN,NaN,182.0,181.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,2.0,0.0,0.0,52.0,1.24,0.0,4.0,20.0,2.0,1.0,51.931080,3.0,3.0,1.0,55313.014168,1.0,75296.121197,44.072913,30.611291,5.652702,4.488967,10.617983,1.0,29.0,174.0,119.0,39.0,86.0,25.49,2.0,5.5,0.0,133.333333,84.000000,0.0,0.0,0.0,0.0,1.0,0.0,130.000000,5531.301417,9412.015150,2.810954,5.394481,-1.416078,5.596735,3.994698,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2704.0,15.083333,52.0,67.083333,1.0,2.0,1.0,0.0,1.0
2,21012.0,1.0,7.0,1.0,3.0,0.0,1.0,1.0,b'001',0.0,0.0,127.0,126.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,2.0,1.0,0.0,63.0,0.89,0.0,4.0,20.0,1.0,2.0,38.637620,2.0,4.0,1.0,18966.755482,1.0,29690.563714,46.390303,29.339153,4.508485,10.484242,10.935122,2.0,33.0,191.0,92.0,78.0,111.0,19.60,2.0,4.8,0.0,122.666667,64.666667,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,1896.675548,3711.320464,2.694137,5.678128,-3.307332,4.463846,2.972125,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,3969.0,10.500000,63.0,73.500000,1.0,0.0,1.0,0.0,1.0
3,21015.0,1.0,6.0,1.0,3.0,0.0,1.0,1.0,b'001',0.0,0.0,25.0,24.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,2.0,1.0,2.0,83.0,1.20,0.0,2.0,0.0,1.0,1.0,67.249329,3.0,1.0,1.0,16558.465868,1.0,12824.553561,48.060086,40.101046,6.171980,5.366953,0.000000,2.0,33.0,141.0,34.0,94.0,60.0,28.32,2.0,5.0,1.0,154.000000,54.000000,0.0,0.0,0.0,1.0,1.0,0.0,38.666667,1655.846587,1603.069195,3.682373,5.882507,-1.693045,6.110872,5.173025,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,6889.0,2.000000,83.0,85.000000,1.0,2.0,1.0,1.0,1.0
4,21017.0,1.0,3.0,2.0,1.0,0.0,1.0,0.0,NaN,NaN,NaN,202.0,201.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,1.0,0.0,37.0,0.21,0.0,3.0,3.0,2.0,3.0,46.395227,1.0,1.0,1.0,11616.087348,1.0,9651.202301,53.498019,35.075704,6.260343,3.491417,0.000000,2.0,42.0,184.0,77.0,65.0,222.0,19.34,2.0,5.1,0.0,102.666667,67.333333,0.0,0.0,0.0,0.0,0.0,0.0,11.333333,1161.608735,1206.400288,3.220909,6.548105,-1.101393,6.198360,3.568864,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1369.0,16.750000,37.0,53.750000,1.0,0.0,1.0,0.0,0.0


In [24]:
# Display total number of rows (i.e., participants) in the dataset
print(f"Total sample size: {df.shape[0]:,}")

# Check the min and max age
min_age = df['RIDAGEYR'].min()
max_age = df['RIDAGEYR'].max()
print(f"Age range: {min_age} to {max_age} years")



Total sample size: 38,900
Age range: 20.0 to 85.0 years


<h2>Merge datasets step-by-step and track sample size</h2>

<h3>Merge demo</h3>

In [367]:
# Step 1: Set file path for NHANES demographic data
folder_path = "/Users/dengshuyue/Desktop/SDOH/analysis/data/less_important"

# Step 2: NHANES demographic files to include (2005–2018 cycles only)
demo_files = [
    "demo_d",  # 2005–2006
    "demo_e",  # 2007–2008
    "demo_f",  # 2009–2010
    "demo_g",  # 2011–2012
    "demo_h",  # 2013–2014
    "demo_i",  # 2015–2016
    "demo_j"   # 2017–2018
]

# Step 3: Load and append files if they exist
demo_dfs = []
for f in demo_files:
    file_path = os.path.join(folder_path, f"{f}.sas7bdat")
    if os.path.exists(file_path):
        df = pd.read_sas(file_path)
        demo_dfs.append(df)
    else:
        print(f"⚠️ File not found: {file_path}")

# Step 4: Combine all loaded demographic datasets
demoall = pd.concat(demo_dfs, ignore_index=True).copy()

# Step 5: Filter to adults age 20+
demoall = demoall[demoall["RIDAGEYR"] >= 20]

# Step 6: Recode marital status from DMDMARTL
conditions = [
    demoall['DMDMARTL'].isin([1, 6]),  # Married or living with partner
    demoall['DMDMARTL'].isin([3, 4]),  # Widowed or divorced
    demoall['DMDMARTL'] == 2,          # Never married
    demoall['DMDMARTL'] == 5           # Separated
]
choices = [1, 2, 3, 4]
demoall['marriage'] = np.select(conditions, choices, default=pd.NA)

# Step 7: Keep only relevant variables
# demoall = demoall[['SEQN', 'marriage', 'SDDSRVYR']]

demoall = demoall[['SEQN', 'RIDAGEYR', 'marriage', 'SDDSRVYR']]


# Step 8: Summary
print(f"✅ Total rows (age 20+): {demoall.shape[0]}")
print(f"✅ Unique SEQN values: {demoall['SEQN'].nunique()}")

# Optional: Preview first few rows
# print(demoall.head())

# Step 9: Save cleaned demoall for reuse
demoall.to_pickle("/Users/dengshuyue/Desktop/SDOH/analysis/data/demoall.pkl")  # Fast & preserves types
# Or as CSV if needed:
# demoall.to_csv("/Users/dengshuyue/Desktop/SDOH/analysis/data/demoall.csv", index=False)



✅ Total rows (age 20+): 39749
✅ Unique SEQN values: 39749


In [331]:
# Step 1: Merge in both age and SDDSRVYR from demoall
mort_with_demo = mort.merge(
    demoall[["SEQN", "RIDAGEYR", "SDDSRVYR"]], on="SEQN", how="left"
)

# Step 2: Filter for adults age ≥ 20
mort_with_demo = mort_with_demo[mort_with_demo["RIDAGEYR"] >= 20]

# Step 3: Check NHANES cycles among adults 20+ with mortality data
print("📊 NHANES cycles in mortality data (age 20+):")
print(mort_with_demo["SDDSRVYR"].value_counts().sort_index())

# 📊 NHANES cycles in mortality data (age 20+)
cycle_counts = mort_with_demo["SDDSRVYR"].value_counts().sort_index()
print("📊 NHANES cycles in mortality data (age 20+):")
print(cycle_counts)

# Print total count
total_count = cycle_counts.sum()
print(f"Total valid mortality count (age 20+): {total_count}")




📊 NHANES cycles in mortality data (age 20+):
SDDSRVYR
4.0     4977
5.0     5926
6.0     6201
7.0     5546
8.0     5757
9.0     5701
10.0    5524
Name: count, dtype: int64
📊 NHANES cycles in mortality data (age 20+):
SDDSRVYR
4.0     4977
5.0     5926
6.0     6201
7.0     5546
8.0     5757
9.0     5701
10.0    5524
Name: count, dtype: int64
Total valid mortality count (age 20+): 39632


<h3>Preprocess SDOH</h3>

In [368]:

# --- Employment data (OCQ) ---
ocq = pd.read_sas(f"{folder_path}/ocq.sas7bdat")

ocq['employ'] = np.nan
ocq.loc[ocq['OCD150'] == 1, 'employ'] = 1
ocq.loc[(ocq['OCD150'] == 3) | (ocq['OCQ380'] == 5), 'employ'] = 2
ocq.loc[ocq['OCQ380'] == 3, 'employ'] = 3
ocq.loc[ocq['OCQ380'].isin([4, 6]), 'employ'] = 4
ocq.loc[ocq['OCQ380'].isin([1, 2, 7]), 'employ'] = 5
ocq['unemployment'] = (ocq['employ'] == 2).astype(int)
ocq = ocq[['SEQN', 'employ', 'unemployment']]

# --- Housing data (HOQ) ---
hoq = pd.read_sas(f"{folder_path}/hoq.sas7bdat")
hoq.loc[hoq['HOQ065'].isin([7, 9]), 'HOQ065'] = np.nan
hoq = hoq[['SEQN', 'HOD050', 'HOQ065']]

# --- Health insurance (HIQs) ---
hiqs = pd.read_sas(f"{folder_path}/hiqs.sas7bdat")
ins = pd.DataFrame({'SEQN': hiqs['SEQN']})
ins['ins'] = np.nan

# Private insurance
ins.loc[(hiqs['HIQ031A'] == 14) | (hiqs['HID030A'] == 1), 'ins'] = 1

# Medicare
ins.loc[
    ((hiqs['HIQ031B'] == 15) & (hiqs['HIQ031D'] != 17) & (hiqs['HIQ031E'] != 18)) |
    ((hiqs['HID030B'] == 1) & (hiqs['HID030C'] != 1)),
    'ins'
] = 2

# Medicaid
ins.loc[
    (((hiqs['HIQ031D'] == 17) | (hiqs['HIQ031E'] == 18)) & (hiqs['HIQ031B'] != 15)) |
    ((hiqs['HID030B'] != 1) & (hiqs['HID030C'] == 1)),
    'ins'
] = 3
ins.loc[
    ((hiqs['HIQ031B'] == 15) & (hiqs['HIQ031D'] == 17)) |
    ((hiqs['HID030B'] == 1) & (hiqs['HID030C'] == 1)),
    'ins'
] = 3

# Other insurance
ins.loc[
    (hiqs[['HIQ031C', 'HIQ031F', 'HIQ031G', 'HIQ031H', 'HIQ031I']].eq(1).any(axis=1)) |
    (hiqs['HID030D'] == 1),
    'ins'
] = 5

# No insurance
ins.loc[(hiqs['HIQ011'] == 2) | (hiqs['HID010'] == 2), 'ins'] = 0

# --- SNAP and Food Security (FSQS) ---
fsqs = pd.read_sas(f"{folder_path}/fsqs.sas7bdat")
snap = pd.DataFrame({'SEQN': fsqs['SEQN'], 'FSDHH': fsqs['FSDHH']})

snap['SNAP'] = np.nan
snap.loc[fsqs['FSQ165'] == 2, 'SNAP'] = 0
snap.loc[fsqs['FSQ012'] == 1, 'SNAP'] = 1
snap.loc[fsqs['FSQ012'] == 2, 'SNAP'] = 0
snap.loc[fsqs['FSQ171'] == 1, 'SNAP'] = 1
snap.loc[fsqs['FSQ171'] == 2, 'SNAP'] = 0
snap.loc[fsqs['FSD170N'] >= 1, 'SNAP'] = 1
snap.loc[fsqs['FSQ170'] == 1, 'SNAP'] = 1
snap.loc[(fsqs['FSQ170'] == 2) & (fsqs['FSD170N'] < 1), 'SNAP'] = 0
snap.loc[fsqs['FSD200'] == 1, 'SNAP'] = 1

# Food Security
snap['FS'] = np.nan
snap.loc[fsqs['FSDHH'].isin([1, 2]), 'FS'] = 1
snap.loc[fsqs['FSDHH'] > 2, 'FS'] = 0

# Keep only final columns
snap = snap[['SEQN', 'SNAP', 'FSDHH', 'FS']]


In [370]:
import pandas as pd
import numpy as np
import os

# Step 0: Setup
folder_path = "/Users/dengshuyue/Desktop/SDOH/analysis/data/less_important"

# Step 1: Load demoall
demoall = pd.read_pickle("/Users/dengshuyue/Desktop/SDOH/analysis/data/demoall.pkl")

if "SEQN" not in demoall.columns or "RIDAGEYR" not in demoall.columns:
    raise ValueError("demoall is missing SEQN or RIDAGEYR")

In [371]:
# Step 2: Filter demoall to adults (age ≥ 20)
age_df = demoall[["SEQN", "RIDAGEYR"]].dropna()
age_df = age_df[age_df["RIDAGEYR"] >= 20]  # ✅ universal filter

# Helper: Filter any df to age ≥ 20 using demoall
def filter_adults(df):
    return df.merge(age_df, on="SEQN", how="inner")

In [372]:
# Step 3: Employment (OCQ)
ocq = filter_adults(pd.read_sas(f"{folder_path}/ocq.sas7bdat"))
ocq['employ'] = np.nan
ocq.loc[ocq['OCD150'] == 1, 'employ'] = 1
ocq.loc[(ocq['OCD150'] == 3) | (ocq['OCQ380'] == 5), 'employ'] = 2
ocq.loc[ocq['OCQ380'] == 3, 'employ'] = 3
ocq.loc[ocq['OCQ380'].isin([4, 6]), 'employ'] = 4
ocq.loc[ocq['OCQ380'].isin([1, 2, 7]), 'employ'] = 5
ocq['unemployment'] = (ocq['employ'] == 2).astype(int)
ocq = ocq[['SEQN', 'employ', 'unemployment']]

# Step 4: Housing (HOQ)
hoq = filter_adults(pd.read_sas(f"{folder_path}/hoq.sas7bdat"))
hoq.loc[hoq['HOQ065'].isin([7, 9]), 'HOQ065'] = np.nan
hoq = hoq[['SEQN', 'HOD050', 'HOQ065']]

# Step 5: Insurance (HIQS)
hiqs = filter_adults(pd.read_sas(f"{folder_path}/hiqs.sas7bdat"))
ins = pd.DataFrame({'SEQN': hiqs['SEQN']})
ins['ins'] = np.nan
ins.loc[(hiqs['HIQ031A'] == 14) | (hiqs['HID030A'] == 1), 'ins'] = 1
ins.loc[((hiqs['HIQ031B'] == 15) & (hiqs['HIQ031D'] != 17) & (hiqs['HIQ031E'] != 18)) |
        ((hiqs['HID030B'] == 1) & (hiqs['HID030C'] != 1)), 'ins'] = 2
ins.loc[(((hiqs['HIQ031D'] == 17) | (hiqs['HIQ031E'] == 18)) & (hiqs['HIQ031B'] != 15)) |
        ((hiqs['HID030B'] != 1) & (hiqs['HID030C'] == 1)), 'ins'] = 3
ins.loc[((hiqs['HIQ031B'] == 15) & (hiqs['HIQ031D'] == 17)) |
        ((hiqs['HID030B'] == 1) & (hiqs['HID030C'] == 1)), 'ins'] = 3
ins.loc[(hiqs[['HIQ031C', 'HIQ031F', 'HIQ031G', 'HIQ031H', 'HIQ031I']].eq(1).any(axis=1)) |
        (hiqs['HID030D'] == 1), 'ins'] = 5
ins.loc[(hiqs['HIQ011'] == 2) | (hiqs['HID010'] == 2), 'ins'] = 0

# Step 6: SNAP & Food Security (FSQS)
fsqs = filter_adults(pd.read_sas(f"{folder_path}/fsqs.sas7bdat"))
snap = pd.DataFrame({'SEQN': fsqs['SEQN'], 'FSDHH': fsqs['FSDHH']})
snap['SNAP'] = np.nan
snap.loc[fsqs['FSQ165'] == 2, 'SNAP'] = 0
snap.loc[fsqs['FSQ012'] == 1, 'SNAP'] = 1
snap.loc[fsqs['FSQ012'] == 2, 'SNAP'] = 0
snap.loc[fsqs['FSQ171'] == 1, 'SNAP'] = 1
snap.loc[fsqs['FSQ171'] == 2, 'SNAP'] = 0
snap.loc[fsqs['FSD170N'] >= 1, 'SNAP'] = 1
snap.loc[fsqs['FSQ170'] == 1, 'SNAP'] = 1
snap.loc[(fsqs['FSQ170'] == 2) & (fsqs['FSD170N'] < 1), 'SNAP'] = 0
snap.loc[fsqs['FSD200'] == 1, 'SNAP'] = 1
snap['FS'] = np.nan
snap.loc[fsqs['FSDHH'].isin([1, 2]), 'FS'] = 1
snap.loc[fsqs['FSDHH'] > 2, 'FS'] = 0
snap = snap[['SEQN', 'SNAP', 'FSDHH', 'FS']]

In [375]:
# Step 1: Read dietary score data
scores = pd.read_excel(os.path.join(folder_path, "i.scores.xlsx"), engine="openpyxl")

# Rename columns to match desired output
scores = scores.rename(columns={
    "seqn": "SEQN",
    "i.FCS": "i_FCS",
    "i.optup": "i_optup",  # keep lowercase here first
    "i.HSR": "i_HSR",
    "i.nutri": "i_nutri"
})

# Then copy and rename for output
scores2 = scores[["SEQN", "i_FCS", "i_optup", "i_HSR", "i_nutri"]].copy()
scores2 = scores2.rename(columns={"i_optup": "i_Optup"})
scores2 = scores2.sort_values("SEQN")


# Step 2: Read covariates from Lu paper
covar = pd.read_sas(os.path.join(folder_path, "covar.sas7bdat"), format="sas7bdat")
covar = covar.rename(columns=str.upper)  # make all column names uppercase to match SAS style
# filter available variables only
covar_vars = ["SEQN", "RIDAGEYR", "SEX", "RACE", "EDU", "INDFMPIR", "SMK_AVG", "SMK_PAST",
              "SMK", "ALCG2", "HEI2015_TOTAL_SCORE", "DIABE"]
covar = covar[[col for col in covar_vars if col in covar.columns]].copy()
covar = covar.sort_values("SEQN")

# Step 3: Read covariates from Meghan paper
covariates1_raw = pd.read_csv(os.path.join(folder_path, "covariates.csv"))
covariates1 = covariates1_raw.rename(columns={"seqn": "SEQN"})
covariates_vars = ["SEQN", "sdmvpsu", "sdmvstra", "met_hr", "perE_alco", "dm_self",
                   "tchol", "hdl", "ldl", "tg", "bmi", "CVD", "dm_rx", "chol_rx",
                   "angina_rx", "lung_disease", "angina", "hba1c", "sbp", "dbp", "cancer"]
covariates1 = covariates1[[col for col in covariates_vars if col in covariates1.columns]].copy()
covariates1 = covariates1.sort_values("SEQN")

# Step 4: Read dietary weight data (filter DAYS == 1)
dietwt = pd.read_sas(os.path.join(folder_path, "gg.sas7bdat"), format="sas7bdat")

# Check for expected columns
required_cols = ["SEQN", "DAYS", "WTDRD1", "WTDR2D", "DR12DRST"]
missing = [col for col in required_cols if col not in dietwt.columns]
if missing:
    print(f"Warning: Missing columns from gg.sas7bdat: {missing}")

# Filter and select
dietwt = dietwt[dietwt["DAYS"] == 1][["SEQN", "WTDRD1", "WTDR2D", "DR12DRST"]].copy()
dietwt = dietwt.sort_values("SEQN")


# Step 5: Read mortality data
mort = pd.read_sas(os.path.join(folder_path, "mortality9918.sas7bdat"), format="sas7bdat")
mort = mort.sort_values("SEQN")

def summarize_df(name, df):
    print(f"{name}:")
    print(f"  Rows: {df.shape[0]}")
    print(f"  Unique SEQN: {df['SEQN'].nunique()}")
    print("-" * 40)

summarize_df("scores2", scores2)
summarize_df("covar", covar)
summarize_df("covariates1", covariates1)
summarize_df("dietwt", dietwt)
summarize_df("mort", mort)

scores2:
  Rows: 39746
  Unique SEQN: 39746
----------------------------------------
covar:
  Rows: 101316
  Unique SEQN: 101316
----------------------------------------
covariates1:
  Rows: 39262
  Unique SEQN: 39262
----------------------------------------
dietwt:
  Rows: 88413
  Unique SEQN: 88413
----------------------------------------
mort:
  Rows: 59064
  Unique SEQN: 59064
----------------------------------------


In [387]:
print("RIDAGEYR" in demoall.columns)  # Should print True
print(demoall.columns)

True
Index(['SEQN', 'RIDAGEYR', 'marriage', 'SDDSRVYR'], dtype='object')


In [388]:
# Ensure SEQN is int in all datasets
for name, df in datasets.items():
    df["SEQN"] = df["SEQN"].astype(int)

In [389]:
# Rename RIDAGEYR in covar to avoid overwriting demoall's RIDAGEYR
covar = covar.rename(columns={"RIDAGEYR": "RIDAGEYR_covar"})


In [390]:
from functools import reduce

# ✅ Step 0: Merge all datasets by SEQN
merge_list = [demoall, hoq, ocq, mort, snap, ins, covar, dietwt, scores2, covariates1]
score_mort = reduce(lambda left, right: pd.merge(left, right, on="SEQN", how="left"), merge_list)

In [391]:
# ✅ Step 1: Filter to adults age ≥ 20
if "RIDAGEYR" not in score_mort.columns:
    raise ValueError("RIDAGEYR is missing after merge — check if demoall was merged correctly.")
score_mort = score_mort[score_mort["RIDAGEYR"] >= 20]

In [392]:
# Step 2: Variable transformations
score_mort["wt10"] = score_mort["WTDRD1"] / 10
score_mort["wt"] = score_mort["WTDR2D"] / 8
score_mort["i_FCS_sd"] = score_mort["i_FCS"] / 10.89
score_mort["i_Optup_sd"] = score_mort["i_Optup"] / 8.17
score_mort["i_nutri_sd"] = -score_mort["i_nutri"] / 3.17
score_mort["i_HSR_sd"] = score_mort["i_HSR"] / 1.01
score_mort["hei2015_sd"] = score_mort["HEI2015_TOTAL_SCORE"] / 13

# Step 3: Recode death indicators
for cause, code in {
    "death_heart": "001", "death_cancer": "002", "death_resp": "003", "Death_inj": "004",
    "death_cerev": "005", "Death_alz": "006", "death_diabe": "007",
    "Death_infl": "008", "Death_kid": "009", "death_other1": "010"
}.items():
    score_mort[cause] = (score_mort["UCOD_LEADING"] == code).astype(int)

# Step 4: Composite categories
score_mort["Death_other"] = score_mort[["death_resp", "Death_inj", "Death_alz", "Death_infl"]].sum(axis=1).clip(upper=1)
score_mort["Death_oth2"] = score_mort[["death_resp", "Death_inj", "Death_alz", "Death_infl", "death_other1"]].sum(axis=1).clip(upper=1)
score_mort["death_cvd"] = score_mort[["death_heart", "death_cerev"]].sum(axis=1).clip(upper=1)
score_mort["death_cmd"] = score_mort[["death_heart", "death_cerev", "death_diabe"]].sum(axis=1).clip(upper=1)
score_mort["death_cmdk"] = score_mort[["death_heart", "death_cerev", "death_diabe", "Death_kid"]].sum(axis=1).clip(upper=1)
score_mort["death_cmdkh"] = score_mort["death_cmdk"]
score_mort.loc[score_mort["DIABETES"] == 1, "death_cmdkh"] = 1
score_mort.loc[score_mort["HYPERTEN"] == 1, "death_cmdkh"] = 1
score_mort["death_cmd"] = score_mort["death_cmd"].fillna(0)
score_mort.loc[score_mort["death_cmd"] == 1, ["Death_other", "Death_oth2"]] = 0

# Step 5: Multiple cause mortality
score_mort["death_multi"] = score_mort["MORTSTAT"]
score_mort.loc[score_mort["death_cmd"] == 1, "death_multi"] = 1
score_mort.loc[score_mort["death_cancer"] == 1, "death_multi"] = 2
score_mort.loc[score_mort["Death_oth2"] == 1, "death_multi"] = 3

# Step 6: Age & time vars
score_mort["agesq"] = score_mort["RIDAGEYR"] ** 2
score_mort["py"] = score_mort["PERMTH_EXM"] / 12
score_mort["agestart"] = score_mort["RIDAGEYR"]
score_mort["ageend"] = score_mort["RIDAGEYR"] + score_mort["py"]

# Step 7: Poverty
score_mort["pir"] = 5
score_mort.loc[(score_mort["INDFMPIR"] < 1.3) & (score_mort["INDFMPIR"].notna()), "pir"] = 1
score_mort.loc[(score_mort["INDFMPIR"] >= 1.3), "pir"] = 2
score_mort.loc[(score_mort["INDFMPIR"] >= 3), "pir"] = 3

# Step 8: Recode SNAP
score_mort.loc[(score_mort["INDFMPIR"].between(0, 1.3)) & (score_mort["SNAP"] != 1), "SNAP"] = 2

# Step 9: BMI categories
score_mort["bmic"] = pd.NA
score_mort.loc[(score_mort["bmi"] > 0) & (score_mort["bmi"] < 18.5), "bmic"] = 0
score_mort.loc[(score_mort["bmi"] >= 18.5) & (score_mort["bmi"] < 25), "bmic"] = 1
score_mort.loc[(score_mort["bmi"] >= 25), "bmic"] = 2
score_mort.loc[(score_mort["bmi"] >= 30), "bmic"] = 3

In [393]:
# check drop by step

# Step 1: Initial inclusion based on ELIGSTAT and MORTSTAT
score_mort["include"] = np.where(
    (score_mort["ELIGSTAT"] == 1) & (score_mort["MORTSTAT"].notna()), 1, 0
)
print(f"After ELIGSTAT & MORTSTAT inclusion: {score_mort['include'].value_counts().to_dict()}")

# Step 2: Exclude missing HEI, or DR12DRST > 1 
step2_cond = (
    (score_mort["include"] == 1) &
    (
        score_mort["HEI2015_TOTAL_SCORE"].isna() |
        #(score_mort["WTDRD1"] <= 0) |
        (score_mort["DR12DRST"] > 1)
    )
)
print(f"Dropped at Step 2 (diet data or recall quality): {step2_cond.sum()}")
score_mort.loc[step2_cond, "include"] = 2

# Step 3: Exclude missing FS, SNAP, or pir == 4
step3_cond = (
    (score_mort["include"] == 1) &
    (
        score_mort["FS"].isna() |
        score_mort["SNAP"].isna() |
        (score_mort["pir"] == 4)
    )
)
print(f"Dropped at Step 3 (missing FS/SNAP or pir=4): {step3_cond.sum()}")
score_mort.loc[step3_cond, "include"] = 3

# Step 4: Exclude if WTDRD1 <= 0 
step4_cond = (score_mort["include"] == 1) & (score_mort["WTDRD1"] <= 0)
print(f"Dropped at Step 4 (zero or negative WTDRD1): {step4_cond.sum()}")
score_mort.loc[step4_cond, "include"] = 4

# Final count of each inclusion code
print(f"📊 Final include flag counts: {score_mort['include'].value_counts().to_dict()}")

# Apply final filter
score_mort = score_mort[score_mort["include"] == 1].copy()
print(f" Final analytic sample size: {score_mort.shape[0]}")


# Inclusion flag
score_mort["include"] = np.where(
    (score_mort["ELIGSTAT"] == 1) & (score_mort["MORTSTAT"].notna()), 1, 0
)
score_mort.loc[
    (score_mort["include"] == 1) &
    (
        (score_mort["HEI2015_TOTAL_SCORE"].isna()) |
        (score_mort["WTDRD1"] <= 0) |
        (score_mort["DR12DRST"] > 1)
    ),
    "include"
] = 2
score_mort.loc[
    (score_mort["include"] == 1) &
    (
        score_mort["FS"].isna() |
        score_mort["SNAP"].isna() |
        (score_mort["pir"] == 4)
    ),
    "include"
] = 3
score_mort.loc[
    (score_mort["include"] == 1) & (score_mort["WTDRD1"] <= 0),
    "include"
] = 4

# Insurance binary
score_mort["ins2"] = np.where(score_mort["ins"] == 0, 0, 1)

# Unemployment indicator
score_mort["unemployment2"] = np.where(score_mort["employ"] > 1, 1, 0)

# Final filter to keep only those with include == 1
score_mort = score_mort[score_mort["include"] == 1].copy()

# Save final dataset
score_mort.to_pickle(os.path.join(folder_path, "SODH_diet_mort.pkl"))  # You can also use .csv or .parquet

After ELIGSTAT & MORTSTAT inclusion: {1: 39632, 0: 117}
Dropped at Step 2 (diet data or recall quality): 4391
Dropped at Step 3 (missing FS/SNAP or pir=4): 596
Dropped at Step 4 (zero or negative WTDRD1): 0
📊 Final include flag counts: {1: 34645, 2: 4391, 3: 596, 0: 117}
 Final analytic sample size: 34645


<h3>check final merged data </h3>

In [394]:
SODH_diet_mort = pd.read_pickle(os.path.join(folder_path, "SODH_diet_mort.pkl"))

score_mort.to_csv("/Users/dengshuyue/Desktop/SDOH/analysis/data/SODH_diet_mort_depr2.csv", index=False)

# Number of rows
total_rows = SODH_diet_mort.shape[0]

# Number of unique SEQN values
unique_ids = SODH_diet_mort['SEQN'].nunique()

print(f" Total rows: {total_rows}")
print(f" Unique SEQN values: {unique_ids}")


 Total rows: 34645
 Unique SEQN values: 34645


In [395]:
# Check age range
min_age = score_mort["RIDAGEYR"].min()
max_age = score_mort["RIDAGEYR"].max()

print(f" Age range: {min_age} to {max_age}")

 Age range: 20.0 to 85.0


In [304]:
# View column names
print(SODH_diet_mort.columns.tolist())

['SEQN', 'marriage', 'SDDSRVYR', 'HOD050', 'HOQ065', 'employ', 'unemployment', 'ELIGSTAT', 'MORTSTAT', 'UCOD_LEADING', 'DIABETES', 'HYPERTEN', 'PERMTH_INT', 'PERMTH_EXM', 'Death_heart', 'Death_cancer', 'Death_resp', 'Death_cerev', 'Death_diabe', 'Death_other', 'death_cvd', 'death_cmd', 'SNAP', 'FSDHH', 'FS', 'ins', 'RIDAGEYR', 'SEX', 'RACE', 'EDU', 'INDFMPIR', 'SMK_AVG', 'SMK', 'ALCG2', 'HEI2015_TOTAL_SCORE', 'DIABE', 'WTDRD1', 'WTDR2D', 'DR12DRST', 'i_FCS', 'i_Optup', 'i_HSR', 'i_nutri', 'sdmvpsu', 'sdmvstra', 'met_hr', 'perE_alco', 'dm_self', 'tchol', 'hdl', 'ldl', 'tg', 'bmi', 'CVD', 'dm_rx', 'chol_rx', 'angina_rx', 'lung_disease', 'angina', 'hba1c', 'sbp', 'dbp', 'cancer', 'wt10', 'wt', 'i_FCS_sd', 'i_Optup_sd', 'i_nutri_sd', 'i_HSR_sd', 'hei2015_sd', 'death_heart', 'death_cancer', 'death_resp', 'Death_inj', 'death_cerev', 'Death_alz', 'death_diabe', 'Death_infl', 'Death_kid', 'death_other1', 'Death_oth2', 'death_cmdk', 'death_cmdkh', 'death_multi', 'agesq', 'py', 'agestart', 'agee

<h1>Demographic Table</h1>

<h2>🧮 Generate Weighted Demographic Summary Table</h2>

<h3>try demo all item </h3>

In [281]:
import pandas as pd

# Load the .pkl file
pkl_path = "/Users/dengshuyue/Desktop/SDOH/analysis/data/SODH_diet_mort.pkl"
df = pd.read_pickle(pkl_path)

# Save as .csv
csv_path = "/Users/dengshuyue/Desktop/SDOH/analysis/data/SODH_diet_mort.csv"
df.to_csv(csv_path, index=False)

print("✅ Conversion complete: .pkl → .csv")


✅ Conversion complete: .pkl → .csv


In [279]:
# Python does not support full survey design (strata + PSU) out of the box
# below code not working 
# === Load data ===
data_path = "/Users/dengshuyue/Desktop/SDOH/analysis/data/"
df = pd.read_pickle(os.path.join(data_path, "SODH_diet_mort.pkl"))

# Drop rows missing survey design variables
df = df.dropna(subset=['wt10', 'sdmvstra', 'sdmvpsu'])

# === Weighted proportions for categorical variables ===
def weighted_props(df, var, weight):
    d = df[[var, weight]].dropna()
    counts = d.groupby(var)[weight].sum()
    total_weight = d[weight].sum()
    props = (counts / total_weight).round(3)
    return counts.round(0).astype(int), (props * 100).round(1)

# === Use Taylor Linearization for SEs of continuous variables ===
from linearmodels.survey import SurveyDesign, SurveyMean

def survey_mean_se(df, var, weight='wt10', strata='sdmvstra', cluster='sdmvpsu'):
    # Remove missing
    d = df[[var, weight, strata, cluster]].dropna()

    # Define survey design
    design = sm.survey.SurveyDesign(
        strata=d[strata],
        cluster=d[cluster],
        weights=d[weight],
    )

    # Fit the design-based estimator
    survey_var = sm.survey.SurveyMean(d[var], design)
    return float(survey_var.mean), float(survey_var.std)

# === Variable dictionaries ===
cat_vars = {
    'SEX': {1: 'Male', 2: 'Female'},
    'RACE': {1: 'Non-Hispanic White', 2: 'Non-Hispanic Black', 3: 'Hispanic', 4: 'Other'},
    'EDU': {1: 'Less than high school', 2: 'High school or equivalent', 3: 'Some college', 4: 'College or above'},
    'pir': {1: '<1.3', 2: '1.3~2.99', 3: '>=3'},
    'SNAP': {0: 'Not participant', 1: 'Participant', 2: 'Income eligible non-participant'},
    'SMK': {1: 'Nonsmokers', 2: 'Former smokers', 3: '<15 cigarettes/day', 4: '15-24.9 cigarettes/day', 5: '≥ 25 cigarettes/day'},
    'ALCG2': {1: 'Nondrinkers', 2: 'Moderate drinker', 3: 'Heavy drinker', 4: 'Missing'},
    'bmic': {1: 'BMI <18.5', 2: '18-24.9', 3: '25-29.9', 4: 'BMI ≥30'}
}

binary_vars = ['DIABETES', 'CVD', 'dm_rx', 'chol_rx', 'angina', 'cancer', 'lung_disease', 'MORTSTAT']
cont_vars = ['RIDAGEYR', 'met_hr', 'bmi', 'hba1c', 'sbp', 'dbp', 'hdl', 'ldl', 'tg', 'HEI2015_TOTAL_SCORE']

# === Generate summary ===
rows = []

# Categorical
for var, mapping in cat_vars.items():
    counts, props = weighted_props(df, var, 'wt10')
    for code, label in mapping.items():
        if code in counts.index:
            rows.append({
                "Variable": var,
                "Category": label,
                "Overall": f"{counts[code]:,.0f} ({props[code]}%)"
            })

# Binary (weighted %)
for var in binary_vars:
    val, se = survey_mean_se(df, var)
    rows.append({
        "Variable": var,
        "Category": "1",
        "Overall": f"{val * 100:.1f}% ({se * 100:.1f})"
    })

# Continuous (mean ± SE)
for var in cont_vars:
    mean, se = survey_mean_se(df, var)
    rows.append({
        "Variable": var,
        "Category": "",
        "Overall": f"{mean:.2f} ({se:.2f})"
    })

# === Create summary table ===
demo_table = pd.DataFrame(rows)

# === Save and preview ===
output_path = os.path.join(data_path, "demo_summary.csv")
demo_table.to_csv(output_path, index=False)

# Show preview
demo_table.head(50)


ModuleNotFoundError: No module named 'linearmodels.survey'

In [264]:

# === Load data ===
df = pd.read_pickle(os.path.join(data_path, "SODH_diet_mort.pkl"))

# Drop missing survey design variables (weight pooled: WTDRD1)
df = df.dropna(subset=['wt10', 'sdmvstra', 'sdmvpsu'])

# === Helper functions ===
def weighted_mean(x, w):
    d = pd.DataFrame({'x': x, 'w': w}).dropna()
    return np.sum(d.x * d.w) / np.sum(d.w)

# not standard 
# def weighted_se(x, w):
#    d = pd.DataFrame({'x': x, 'w': w}).dropna()
#    mean = weighted_mean(d.x, d.w)
#    return np.sqrt(np.sum(d.w * (d.x - mean)**2) / ((len(d) - 1) * np.sum(d.w) / len(d)))

# Only acceptable if weights are uniform
# def weighted_se(x, w):
#    d = pd.DataFrame({'x': x, 'w': w}).dropna()
#    mean = np.sum(d.x * d.w) / np.sum(d.w)
#    var = np.sum(d.w * (d.x - mean)**2) / np.sum(d.w)
#    se = np.sqrt(var / len(d))  # Approximate
#    return se

# 🔥 Check best practice for estimating SE with survey weights

def weighted_se(x, w):
    d = pd.DataFrame({'x': x, 'w': w}).dropna()
    mean = np.sum(d.x * d.w) / np.sum(d.w)
    eff_n = (np.sum(d.w))**2 / np.sum(d.w**2)  # Effective sample size
    var = np.sum(d.w * (d.x - mean)**2) / np.sum(d.w)
    se = np.sqrt(var / eff_n)
    return se

    
def weighted_props(df, var, weight):
    d = df[[var, weight]].dropna()
    counts = d.groupby(var)[weight].sum()
    total_weight = d[weight].sum()
    props = (counts / total_weight).round(3)
    return counts.round(0).astype(int), (props * 100).round(1)

# === Categorical variables ===
cat_vars = {
    'SEX': {1: 'Male', 2: 'Female'},
    'RACE': {1: 'Non-Hispanic White', 2: 'Non-Hispanic Black', 3: 'Hispanic', 4: 'Other'},
    'EDU': {1: 'Less than high school', 2: 'High school or equivalent', 3: 'Some college', 4: 'College or above'},
    'pir': {1: '<1.3', 2: '1.3~2.99', 3: '>=3'},
    'SNAP': {0: 'Not participant', 1: 'Participant', 2: 'Income eligible non-participant'},
    'SMK': {1: 'Nonsmokers', 2: 'Former smokers', 3: '<15 cigarettes/day', 4: '15-24.9 cigarettes/day', 5: '≥ 25 cigarettes/day'},
    'ALCG2': {1: 'Nondrinkers', 2: 'Moderate drinker', 3: 'Heavy drinker', 4: 'Missing'},
    'bmic': {1: 'BMI <18.5', 2: '18-24.9', 3: '25-29.9', 4: 'BMI ≥30'}
}

rows = []

for var, mapping in cat_vars.items():
    counts, props = weighted_props(df, var, 'wt10')
    for code, label in mapping.items():
        if code in counts.index:
            rows.append({
                "Variable": var,
                "Category": label,
                "Overall": f"{counts[code]:,.0f} ({props[code]}%)"
            })

# === Binary variables ===
binary_vars = ['DIABETES', 'CVD', 'dm_rx', 'chol_rx', 'angina', 'cancer', 'lung_disease', 'MORTSTAT']
for var in binary_vars:
    val = weighted_mean(df[var].fillna(0), df['wt10']) * 100
    rows.append({
        "Variable": var,
        "Category": "1",
        "Overall": f"{val:.1f}%"
    })

# === Continuous variables ===
cont_vars = ['RIDAGEYR', 'met_hr', 'bmi', 'hba1c', 'sbp', 'dbp', 'hdl', 'ldl', 'tg', 'HEI2015_TOTAL_SCORE']
for var in cont_vars:
    mean = weighted_mean(df[var], df['wt10'])
    se = weighted_se(df[var], df['wt10'])
    rows.append({
        "Variable": var,
        "Category": "",
        "Overall": f"{mean:.2f} ({se:.2f})"
    })

# === Create summary table ===
demo_table = pd.DataFrame(rows)

# Save, preview

# Optionally: save to CSV
# demo_table.to_csv("demo_summary.csv", index=False)
demo_table.to_csv(output_path, index=False)

demo_table.head(50)

,Variable,Category,Overall
0,SEX,Male,"82,147,536 (47.8%)"
1,SEX,Female,"89,889,015 (52.2%)"
2,RACE,Non-Hispanic White,"117,066,666 (68.0%)"
3,RACE,Non-Hispanic Black,"19,270,383 (11.2%)"
4,RACE,Hispanic,"23,166,597 (13.5%)"
5,RACE,Other,"12,532,906 (7.3%)"
6,EDU,Less than high school,"27,365,333 (15.9%)"
7,EDU,High school or equivalent,"41,028,422 (23.9%)"
8,EDU,Some college,"54,476,932 (31.7%)"
9,EDU,College or above,"49,075,426 (28.5%)"


In [227]:
# Clean data: drop rows with missing mortality status or weight
mort_df = df[['MORTSTAT', 'WTDRD1']].dropna()

# Calculate total weighted sum
total_weight = mort_df['WTDRD1'].sum()

# Weighted percentage of deceased (MORTSTAT == 1)
dead_weight = mort_df.loc[mort_df['MORTSTAT'] == 1, 'WTDRD1'].sum()
weighted_pct_dead = (dead_weight / total_weight) * 100

print(f"Weighted percentage deceased: {weighted_pct_dead:.1f}%")


Weighted percentage deceased: 9.1%


In [22]:
# Define unweighted counts and proportions
def unweighted_props(df, var):
    counts = df[var].value_counts(dropna=False).sort_index()
    props = counts / counts.sum() * 100
    return counts, props.round(1)

# Now use it for 'sex'
counts, props = unweighted_props(df, 'sex')

# Optionally map codes to labels
sex_labels = {1: 'Male', 2: 'Female'}
for val in counts.index:
    label = sex_labels.get(val, val)
    print(f"{label}: {counts[val]} ({props[val]}%)")
18530+19890


Male: 18845 (48.4%)
Female: 20055 (51.6%)


38420

In [ ]:
with open('/Users/dengshuyue/Desktop/SDOH/analysis/code/Ref/2.Prepare data_master.sas', 'r', encoding='latin1') as f:
    sas_code = f.read()

print(sas_code[:20000])  # Preview the first X,000 characters

In [ ]:
with open('/Users/dengshuyue/Desktop/SDOH/analysis/code/Ref/2.1_Prepare data_covariates.sas', 'r', encoding='latin1') as f:
    sas_code = f.read()

print(sas_code[:20000])  # Preview the first X,000 characters

In [ ]:
with open('/Users/dengshuyue/Desktop/SDOH/analysis/code/Descriptions_1.sas', 'r', encoding='latin1') as f:
    sas_code = f.read()

print(sas_code[:20000])  # Preview the first 15,000 characters

In [ ]:
# Diabetes Identification Note:
# The final diabetes indicator variable used in the analysis is 'diabe2'.
# This composite variable identifies diabetes based on the following criteria:
# - Self-reported physician diagnosis (DIQ010), current insulin use (DIQ050), or oral medication use (DIQ070)
# - Prescription drug data indicating diabetes treatment (dm_rx2)
# - Fasting glucose ≥ 126 mg/dL (glu_dm)
# - OGTT ≥ 200 mg/dL (ogtt_dm)
# - HbA1c ≥ 6.5% (hb_dm)
# Any one of these criteria being met will set 'diabe2' = 1, otherwise 0.


In [ ]:
with open('/Users/dengshuyue/Desktop/SDOH/analysis/Analysis1_COX_allcause_bysub.sas', 'r', encoding='latin1') as f:
    sas_code = f.read()

print(sas_code[:10000])  # Preview the first 15,000 characters

In [ ]:
with open('/Users/dengshuyue/Desktop/SDOH/analysis/Analysis1_dataprep.sas', 'r', encoding='latin1') as f:
    sas_code = f.read()

print(sas_code[:20000])  # Preview the first 15,000 characters

<h1>SDOH score</h1>

In [20]:
# see housing and employ
df[['HOQ065', 'HOD050', 'EMPLOY']]

,HOQ065,HOD050,EMPLOY
0,1.0,6.0,1.0
1,1.0,3.0,2.0
2,1.0,7.0,3.0
3,1.0,6.0,3.0
4,2.0,3.0,1.0
...,...,...,...
38895,3.0,8.0,1.0
38896,1.0,6.0,3.0
38897,2.0,1.0,1.0
38898,2.0,8.0,2.0


In [21]:
# List of selected SDOH-related variables
selected_cols = ['HOQ065', 'HOD050', 'MOR']

# Summary function for each variable
for col in selected_cols:
    print(f"\n=== {col} Summary ===")
    print("Value counts:")
    print(df[col].value_counts(dropna=False))
    print("\nProportions:")
    print((df[col].value_counts(normalize=True, dropna=False) * 100).round(2))
    print(f"\nMissing values: {df[col].isnull().sum()}")


=== HOQ065 Summary ===
Value counts:
HOQ065
1.0    24162
2.0    13779
3.0      907
NaN       52
Name: count, dtype: int64

Proportions:
HOQ065
1.0    62.11
2.0    35.42
3.0     2.33
NaN     0.13
Name: proportion, dtype: float64

Missing values: 52

=== HOD050 Summary ===
Value counts:
HOD050
5.0      8034
6.0      7469
4.0      6416
7.0      5175
8.0      3352
3.0      3193
9.0      1908
10.0     1122
2.0       820
11.0      476
12.0      285
1.0       281
13.0      260
999.0      43
777.0      39
NaN        27
Name: count, dtype: int64

Proportions:
HOD050
5.0      20.65
6.0      19.20
4.0      16.49
7.0      13.30
8.0       8.62
3.0       8.21
9.0       4.90
10.0      2.88
2.0       2.11
11.0      1.22
12.0      0.73
1.0       0.72
13.0      0.67
999.0     0.11
777.0     0.10
NaN       0.07
Name: proportion, dtype: float64

Missing values: 27

=== EMPLOY Summary ===
Value counts:
EMPLOY
1.0    20703
3.0     7765
5.0     4323
4.0     3634
2.0     1594
NaN      881
Name: count, dtype: